In [16]:
import pandas as pd
import numpy as np
from copy import deepcopy

In [ ]:
def create_ET_TIME(df: pd.DataFrame):
    assert df.index.names == ["TS", "EXTRA_KEY", "ID_QI"]
    df_2 = deepcopy(df)
    del df
    df_2["UTC_TIME"] = pd.to_datetime(df_2.index.get_level_values(level="TS"))
    df_2["ET_TIME"] = df_2["UTC_TIME"].dt.tz_localize('UTC').dt.tz_convert('US/Eastern')
    df_2["ET_DATE"] = df_2["ET_TIME"].dt.date
    df_2["ET_HOUR"] = df_2["ET_TIME"].dt.hour
    df_2["ET_MINUTE"] = df_2["ET_TIME"].dt.minute
    df_2["TOMORROW_ET_DATE"] = df_2["ET_DATE"] + pd.Timedelta(days=1)
    df_2["IS_INTRADAY"] = (((df_2["ET_HOUR"]==9) & df_2["ET_MINUTE"].between(30, 59)) | df_2["ET_HOUR"].between(10, 15)).astype(int)
    df_2["IS_BEFORE_INTRADAY"] = (df_2["ET_HOUR"].between(0, 8) | ((df_2["ET_HOUR"]==9) & df_2["ET_MINUTE"].between(0, 29))).astype(int)
    df_2["IS_AFTER_INTRADAY"] = df_2["ET_HOUR"].between(16, 23).astype(int)
    df_2["C2C_ET_DATE"] = np.where(
        df_2["IS_BEFORE_INTRADAY"] + df_2["IS_INTRADAY"] == 1, 
        df_2["ET_DATE"],
        np.where(df_2["IS_AFTER_INTRADAY"] == 1, df_2["TOMORROW_ET_DATE"], None)
    )
    return df_2


def agg_per_id_qi_per_day(df: pd.DataFrame, features: list[str], date_name="C2C_ET_DATE", intraday_col="IS_INTRADAY", min_volume=0, min_intraday_volume=0, agg='mean'):
    df_2 = create_volumes(df, date_name, intraday_col, min_volume, min_intraday_volume)

    mask_volume = (df_2["C2C_VOLUME"] >= min_volume) & (df_2["INTRADAY_VOLUME"] >= min_intraday_volume)
    df_2 = df_2[mask_volume]

    data_agg_id_qi_d = df_2.groupby([date_name, "ID_QI"])[features].agg(agg)
    return data_agg_id_qi_d

def create_volumes(df: pd.DataFrame, date_name="C2C_ET_DATE", intraday_col="IS_INTRADAY"):
    df_2 = deepcopy(df)
    del df
    assert df_2.index.names == [None]
    assert df_2[intraday_col].isin([0, 1, None, np.nan]).all()
    features = features + ["C2C_VOLUME", "INTRADAY_VOLUME"]
    
    df_2.dropna(subset=date_name, inplace=True)
    df_2.set_index([date_name, "ID_QI"], inplace=True)
    df_2.sort_index(inplace=True)
    df_2_group = df_2.groupby([date_name, "ID_QI"])
    df_2["C2C_VOLUME"] = df_2_group.size()
    df_2["INTRADAY_VOLUME"] = df_2_group[intraday_col].sum()

    return df_2

In [56]:
ravenpack_tmsr = create_ET_TIME(pd.read_excel("data.xlsx").set_index(["TS", "EXTRA_KEY", "ID_QI"]))
display(ravenpack_tmsr.head(2))

features = ["TMSR_CSS", "TMSR_ESS", "IS_INTRADAY"]
ravenpack_tmsr = ravenpack_tmsr.reset_index()[["C2C_ET_DATE", "ID_QI"] + features]
display(ravenpack_tmsr.head(2))
data_agg_id_qi_d = agg_per_id_qi_per_day(ravenpack_tmsr, features)

,,,TMSR_CSS,TMSR_ESS,UTC_TIME,ET_TIME,ET_DATE,ET_HOUR,ET_MINUTE,TOMORROW_ET_DATE,IS_INTRADAY,IS_BEFORE_INTRADAY,IS_AFTER_INTRADAY,C2C_ET_DATE
TS,EXTRA_KEY,ID_QI,,,,,,,,,,,,
2010-01-01 03:00:00,EK1,5,0.5,0.10,2010-01-01 03:00:00,2009-12-31 22:00:00-05:00,2009-12-31,22,0,2010-01-01,0,0,1,2010-01-01
2010-01-01 05:00:00,EK2,7,0.3,0.02,2010-01-01 05:00:00,2010-01-01 00:00:00-05:00,2010-01-01,0,0,2010-01-02,0,1,0,2010-01-01


,C2C_ET_DATE,ID_QI,TMSR_CSS,TMSR_ESS,IS_INTRADAY
0,2010-01-01,5,0.5,0.10,0
1,2010-01-01,7,0.3,0.02,0


In [57]:
ravenpack_tmsr

,C2C_ET_DATE,ID_QI,TMSR_CSS,TMSR_ESS,IS_INTRADAY
0,2010-01-01,5,0.5,0.10,0
1,2010-01-01,7,0.3,0.02,0
2,2010-01-01,5,-0.1,-0.14,0
3,2010-01-01,5,-0.2,-0.18,0
4,2010-01-01,7,0.1,-0.06,1
5,2010-01-03,5,0.2,-0.02,0
6,2010-01-03,5,0.1,-0.06,0
7,2010-01-03,7,-0.1,-0.14,0
8,2010-01-03,7,-0.4,-0.26,1
9,2010-01-04,7,0.8,0.22,0


In [58]:
data_agg_id_qi_d

TMSR_CSS  TMSR_ESS  IS_INTRADAY  C2C_VOLUME  \
C2C_ET_DATE ID_QI                                                
2010-01-01  5      0.066667 -0.073333          0.0         3.0   
            7      0.200000 -0.020000          0.5         2.0   
2010-01-03  5      0.150000 -0.040000          0.0         2.0   
            7     -0.250000 -0.200000          0.5         2.0   
2010-01-04  7      0.450000  0.080000          0.0         2.0   
2010-01-05  5      0.600000  0.140000          1.0         1.0   
            7     -0.200000 -0.180000          1.0         1.0   
2010-01-06  5     -0.700000 -0.380000          0.0         1.0   

                   INTRADAY_VOLUME  
C2C_ET_DATE ID_QI                   
2010-01-01  5                  0.0  
            7                  1.0  
2010-01-03  5                  0.0  
            7                  1.0  
2010-01-04  7                  0.0  
2010-01-05  5                  1.0  
            7                  1.0  
2010-01-06  5                  0.0

In [59]:
ravenpack_tmsr.groupby(["C2C_ET_DATE", "ID_QI"]).size()

C2C_ET_DATE  ID_QI
2010-01-01   5        3
             7        2
2010-01-03   5        2
             7        2
2010-01-04   7        2
2010-01-05   5        1
             7        1
2010-01-06   5        1
dtype: int64